In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import tabulate

from sklearn.model_selection import train_test_split
from imblearn.under_sampling import ClusterCentroids , TomekLinks , NearMiss

from sklearn.metrics import accuracy_score , confusion_matrix , precision_score , recall_score , f1_score , classification_report , auc , roc_curve , ConfusionMatrixDisplay ,  roc_auc_score
from sklearn.utils import resample
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression , LogisticRegression
from sklearn.preprocessing import LabelEncoder , RobustScaler , StandardScaler
from sklearn.neural_network import MLPClassifier

plt.style.use("fivethirtyeight")
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/creditcardfraud/creditcard.csv


# **Obtain the Data Set**

In [3]:
creditCard = pd.read_csv("/kaggle/input/creditcardfraud/creditcard.csv")

In [4]:
creditCard.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [5]:
creditCard.sample(25)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
27906,34760.0,1.308103,-0.622334,0.192360,-0.659395,-1.048954,-0.829602,-0.450705,-0.091093,-1.046131,...,-0.330417,-0.612867,0.020195,0.345680,0.201488,1.023155,-0.081334,-0.002744,35.90,0
33786,37388.0,-1.485307,-0.655850,2.126541,-0.742914,1.115250,-0.707818,-0.424104,0.285210,0.614438,...,0.042305,-0.141361,-0.032943,-0.445851,0.040308,0.030910,0.030021,0.133771,11.50,0
132641,80063.0,-0.242695,-3.816305,-0.873068,-0.716590,-1.777252,0.511419,0.168492,-0.092873,-2.136339,...,0.410029,-0.211709,-0.783269,-0.242389,0.177708,-0.111496,-0.116895,0.139637,864.20,0
23631,32848.0,0.380979,1.302850,-0.440876,2.597059,0.040942,-0.891093,0.469866,0.127332,-1.246157,...,0.155254,0.328737,0.309279,0.369613,-0.605690,-0.064435,-0.309360,-0.208467,18.35,0
178893,123831.0,1.458080,-0.930260,-2.365379,0.447669,0.381869,-0.278734,0.504830,-0.242197,0.927372,...,-0.092675,-0.719976,-0.146016,-0.058131,-0.061501,-0.112025,-0.065275,0.034866,327.81,0
30490,35952.0,1.347869,-0.448233,0.132802,-0.632698,-0.945992,-1.054726,-0.277647,-0.182545,-1.250740,...,-0.381259,-0.687945,0.063524,0.548276,0.239593,0.981676,-0.083848,-0.007045,13.85,0
142996,85059.0,-1.067823,1.310714,2.173327,1.988520,-0.495969,0.370377,-0.008480,0.415188,-0.648187,...,0.253825,0.707896,-0.213369,0.542448,-0.039654,0.013975,-0.619910,-0.231934,8.69,0
232619,147240.0,-0.624072,-1.629162,2.787103,-0.802752,-0.546602,1.055496,-1.044636,-0.156757,1.084087,...,0.211586,1.853723,0.187712,-0.302499,-0.695701,0.045191,-0.328132,-0.515020,1.00,0
142108,84580.0,-0.857991,0.575800,2.468358,-1.976620,-0.470091,-0.725646,0.503118,-0.168531,1.422646,...,-0.040589,0.286015,-0.397080,0.399593,0.546668,-0.799465,0.054984,-0.086499,1.00,0
216993,140720.0,2.038759,-1.032385,-1.869156,-1.150735,-0.105594,-0.465405,-0.242788,-0.134536,-0.781771,...,0.116692,-0.047298,0.118469,0.192812,-0.103412,-0.341225,-0.079340,-0.056289,108.00,0


# **Exploring the Data Set**

In [6]:
creditCard.describe().T

,count,mean,std,min,25%,50%,75%,max
Time,284807.0,9.481386e+04,47488.145955,0.000000,54201.500000,84692.000000,139320.500000,172792.000000
V1,284807.0,1.168375e-15,1.958696,-56.407510,-0.920373,0.018109,1.315642,2.454930
V2,284807.0,3.416908e-16,1.651309,-72.715728,-0.598550,0.065486,0.803724,22.057729
V3,284807.0,-1.379537e-15,1.516255,-48.325589,-0.890365,0.179846,1.027196,9.382558
V4,284807.0,2.074095e-15,1.415869,-5.683171,-0.848640,-0.019847,0.743341,16.875344
V5,284807.0,9.604066e-16,1.380247,-113.743307,-0.691597,-0.054336,0.611926,34.801666
V6,284807.0,1.487313e-15,1.332271,-26.160506,-0.768296,-0.274187,0.398565,73.301626
V7,284807.0,-5.556467e-16,1.237094,-43.557242,-0.554076,0.040103,0.570436,120.589494
V8,284807.0,1.213481e-16,1.194353,-73.216718,-0.208630,0.022358,0.327346,20.007208
V9,284807.0,-2.406331e-15,1.098632,-13.434066,-0.643098,-0.051429,0.597139,15.594995


In [7]:
creditCard.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [8]:
print(f"The Data Set contain {creditCard.shape[0]} Rows & {creditCard.shape[1]} Columns")
print(f"The Data Set contain {creditCard.duplicated().sum()} record duplicated") # Flag Here
print(f"Number of Nulls in each features :\n{creditCard.isna().sum()}")

The Data Set contain 284807 Rows & 31 Columns
The Data Set contain 1081 record duplicated
Number of Nulls in each features :
Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64


# **Modeling Our Data**

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE

X = creditCard.drop('Class', axis=1)
y = creditCard['Class']

smote = SMOTE(random_state=42)
X_resampled_smote, y_resampled_smote = smote.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(
    X_resampled_smote, y_resampled_smote, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [25]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from sklearn.metrics import classification_report, accuracy_score

model = Sequential()

model.add(Dense(512, input_dim=X_train.shape[1], activation='relu'))  
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))

model.add(Dense(len(np.unique(y_train)), activation='softmax')) 

model.compile(optimizer=SGD(learning_rate=0.0005), 
              loss='sparse_categorical_crossentropy',  
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs= 50, batch_size=32, verbose=1)

y_pred_multiclass = np.argmax(model.predict(X_test), axis=1)  

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 18s 1ms/step - accuracy: 0.9099 - loss: 0.2914
Epoch 2/50
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 17s 1ms/step - accuracy: 0.9670 - loss: 0.0924
Epoch 3/50
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 17s 1ms/step - accuracy: 0.9755 - loss: 0.0703
Epoch 4/50
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 17s 1ms/step - accuracy: 0.9800 - loss: 0.0576
Epoch 5/50
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 17s 1ms/step - accuracy: 0.9830 - loss: 0.0496
Epoch 6/50
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 17s 1ms/step - accuracy: 0.9852 - loss: 0.0434
Epoch 7/50
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 17s 1ms/step - accuracy: 0.9871 - loss: 0.0381
Epoch 8/50
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 17s 1ms/step - accuracy: 0.9888 - loss: 0.0337
Epoch 9/50
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 17s 1ms/step - accuracy: 0.9900 - loss: 0.0307
Epoch 10/50
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 17s 1ms/step - accuracy: 0.9914 - loss: 0.0269
Epoch 11/50
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 17s 1ms/step - accuracy: 0.9926 - loss: 0.02

In [26]:
print("Optimized Multiclass Classification Report:")
print(classification_report(y_test, y_pred_multiclass))
print(precision_score(y_test, y_pred_multiclass))
print(f"Accuracy: {accuracy_score(y_test, y_pred_multiclass)}")

Optimized Multiclass Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56750
           1       1.00      1.00      1.00     56976

    accuracy                           1.00    113726
   macro avg       1.00      1.00      1.00    113726
weighted avg       1.00      1.00      1.00    113726

0.9984752891692955
Accuracy: 0.9992086242372017


In [24]:
model.evaluate(X_test,y_test)

3554/3554 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.9922 - loss: 0.0254


[0.025150442495942116, 0.9927193522453308]

In [17]:
model.save("trial1.keras")